In [ ]:
presentation = True

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
if not presentation:
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 24
matplotlib.rcParams['figure.figsize'] = 10, 10
# matplotlib.rcParams['xtick.bottom'] = False
# matplotlib.rcParams['ytick.left'] = False
matplotlib.backend = 'Agg'

In [ ]:
import mud.funs as funs
import mud.util as utils
import mud.plot as plot

In [ ]:
# import importlib
# importlib.reload(plot)

In [ ]:
np.random.seed(24)
numQoI = 10
A = utils.rotationMap(numQoI, orth=False)
A = A.reshape(-1,2)
b = np.zeros((numQoI,1))
# A, b = A[np.array([2,6]),:].reshape(-1,2), b[np.array([2,6])].reshape(-1,1)
ref_param = np.array([0.5, 0.5]).reshape(-1,1)
y = A@ref_param + b
initial_mean = np.random.randn(2).reshape(-1,1)
tol = 0.1
initial_cov = np.eye(2)*utils.std_from_equipment(tol)

In [ ]:
# mud_chain_test = performEpoch(A, b, y, initial_mean, initial_cov, data_cov=None)

In [ ]:
def applystyle(plotobj):
    plotobj.axis('equal')
    plotobj.ylim([-0.8, 1.5])
    plotobj.xlim([-0.5, 1.5])
    # plt.xticks([])
    # plt.yticks([])

In [ ]:
plt.figure(figsize=(10,10))
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=1, idx=None)
plot.plotChain(mud_chain, ref_param)
plot.plot_contours(A, ref_param)

applystyle(plt)

plt.savefig(f'iterative/{numQoI}D-firstepoch.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=1)
plot.plotChain(mud_chain, ref_param)
for _ in range(3):
    mud_chain = funs.iterate(A, b, y, mud_chain[-1], initial_cov, data_cov=None, num_epochs=1)
    plot.plotChain(mud_chain, ref_param)
    
plot.plot_contours(A, ref_param)
applystyle(plt)
plt.savefig(f'iterative/{numQoI}D-fewepochs.png')
plt.show()

# What if we picked out two of the ten rows?

In [ ]:
np.random.seed(21)
plt.figure(figsize=(10,10))
idx_choice = [np.random.randint(numQoI), np.random.randint(numQoI)]
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=5, idx=idx_choice)
plot.plot_contours(A, ref_param, subset=idx_choice, color='k')
plot.plotChain(mud_chain,ref_param)

applystyle(plt)
plt.savefig(f'iterative/{numQoI}D-fewepochs-pair.png')
plt.show()

np.random.seed(19)
idx_choice = [np.random.randint(numQoI), np.random.randint(numQoI)]
plt.figure(figsize=(10,10))
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=5, idx=idx_choice)
plot.plot_contours(A, ref_param, color='b', subset=idx_choice)
plot.plotChain(mud_chain, ref_param)

applystyle(plt)
plt.savefig(f'iterative/{numQoI}D-fewepochs-pair-alt.png')
plt.show()

# Picking orthogonal rows of A leads to immediate convergence

In [ ]:
plt.figure(figsize=(10,10))
idx_choice = [3,8]
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=5, idx=idx_choice)
plot.plot_contours(A, ref_param, color='k', subset=idx_choice)
plot.plotChain(mud_chain, ref_param)

idx_choice = [1,6]
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=5, idx=idx_choice)
plot.plot_contours(A, ref_param, color='b', subset=idx_choice)
plot.plotChain(mud_chain, ref_param, 'b')

applystyle(plt)
plt.savefig(f'iterative/{numQoI}D-firstepoch-pair-smart.png')
plt.show()

np.random.seed(12)
plt.figure(figsize=(10,10))
idx_choice = np.arange(numQoI)
np.random.shuffle(idx_choice)
mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=1, idx=idx_choice)
plot.plot_contours(A[np.array(idx_choice),:], ref_param)
plot.plotChain(mud_chain, ref_param)

applystyle(plt)
plt.savefig(f'iterative/{numQoI}D-firstepoch-rand.png')
plt.show()

---
# Quantifying the Impact of all of "this"

Let us be more formal and perform this experiment for the rotationally-defined $A$ for more choices of initial means.
We also want to experiment with several orderings of indices for our iteration.

In [ ]:
num_trials = 100
model_eval_budget = 100
num_epochs = model_eval_budget//numQoI
# ordered
idx_choice = np.arange(numQoI)
mud_chains_from_ordered = []
for trial in range(num_trials):
    initial_mean = np.random.rand(2,1)
    mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=num_epochs, idx=idx_choice)
    mud_chains_from_ordered.append(mud_chain)

# some random ordering which will be used over multiple epochs
# ordering changes each trial
idx_choice = np.arange(numQoI)
mud_chains_from_shuffles = []
for trial in range(num_trials):
    initial_mean = np.random.rand(2,1)
    np.random.shuffle(idx_choice)
    mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=num_epochs, idx=idx_choice)
    mud_chains_from_shuffles.append(mud_chain)

# uses all the same data same number of times, but only one epoch due to
# the indices being repeated in a list before being shuffled
idx_choice = list(np.arange(numQoI))*num_epochs
mud_chains_from_batch = []
for trial in range(num_trials):
    initial_mean = np.random.rand(2,1)
    np.random.shuffle(idx_choice)
    mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=1, idx=idx_choice)
    mud_chains_from_batch.append(mud_chain)

# no imposed limitation on using data equal number of times by the end
idx_choice = np.random.choice(np.arange(numQoI), size=num_epochs*numQoI)
mud_chains_from_randoms = []
for trial in range(num_trials):
    initial_mean = np.random.rand(2,1)
    np.random.shuffle(idx_choice)
    mud_chain = funs.iterate(A, b, y, initial_mean, initial_cov, data_cov=None, num_epochs=1, idx=idx_choice)
    mud_chains_from_randoms.append(mud_chain)


In [ ]:
plt.figure(figsize=(20,10))
error = []
for mud_chain in mud_chains_from_ordered:
    approx_error = [np.linalg.norm(m - ref_param) for m in mud_chain]
    error.append(approx_error)
    plt.plot(approx_error, 'r', alpha=0.1)
plt.plot(np.mean(np.array(error),axis=0), 'r', lw=5, label='Ordered QoI $(10\\times 10D)$)')

error = []
for mud_chain in mud_chains_from_shuffles:
    approx_error = [np.linalg.norm(m - ref_param) for m in mud_chain]
    error.append(approx_error)
    plt.plot(approx_error, 'k', alpha=0.1)
plt.plot(np.mean(np.array(error),axis=0), 'k', lw=5, label='Shuffled QoI $(10\\times 10D)$')

error = []
for mud_chain in mud_chains_from_batch:
    approx_error = [np.linalg.norm(m - ref_param) for m in mud_chain]
    error.append(approx_error)
    plt.plot(approx_error, 'b', alpha=0.1)
plt.plot(np.mean(np.array(error),axis=0), 'b', lw=5, label='Batch QoI $(1\\times 100D)$')

error = []
for mud_chain in mud_chains_from_randoms:
    approx_error = [np.linalg.norm(m - ref_param) for m in mud_chain]
    error.append(approx_error)
#     plt.plot(approx_error, 'orange', alpha=0.1)
plt.plot(np.mean(np.array(error),axis=0), 'orange', lw=5, label='Random QoI $(1\\times 100D)$')


plt.legend(loc='lower left')
plt.yscale('log')
plt.ylim(1E-16, 9E-1)
# plt.xscale('log')
plt.ylabel("$||\lambda - \lambda^\dagger||$", fontsize=36)
plt.xlabel("Iteration Step", fontsize=32)
plt.savefig(f'iterative/{numQoI}D-convergence-comparison.png')
plt.show()